<a href="https://colab.research.google.com/github/RemiFlicoteaux/hdbstudy/blob/main/scenario_oncology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creation of oncology fictive clinical scenarios for medical documents generation

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Path to repository with references data
path_ref = "/content/drive/MyDrive/Scenario/"

In [13]:
# ICD10 cancer files
cim_cancer = pd.read_excel(path_ref + "REFERENTIEL_METHODE_DIM_CANCER_20140411.xls")
codes_cancer = cim_cancer.CIM10
# ICD10 Cancer metastase
codes_cancer_meta = ["C770","C771","C772","C773","C774","C775","C778","C779",
             "C780","C781","C782","C783","C784","C785","C786","C787","C788",
             "C790","C791","C792","C793","C794","C795","C796","C797","C798"]



racine_chimio = ['28Z07','17M05','17M06']
racine_radio = ["17K04","17K05","17K08","17K09","28Z10","28Z11","28Z18","28Z19",
                "28Z20","28Z21","28Z22","28Z23","28Z24","28Z25"]
racine_greffe = ["27Z02","27Z03","27Z04"]
racine_transfusion = ["28Z14"]
racine_soins_palliatifs = ["23Z02"]
racine_soins_stomies = ["06M17"]
racine_aphereses = ["28Z16"]
racine_deces = ["04M24"]
racine_bilan = ["23M03"]


In [19]:
df_situations = pd.read_excel(path_ref + "situations_cliniques.xlsx")
df_das = pd.read_excel(path_ref + "situations_cliniques_das.xlsx")
df_actes = pd.read_excel(path_ref + "situations_cliniques_actes.xlsx")
df_icd_official = pd.read_excel(path_ref + "cim_2024.xlsx")
df_chronique = pd.read_excel(path_ref + "Affections chroniques.xlsx",header=None,names=["code","c","libelle"])

In [26]:
icd10_chronic = df_chronique.code[df_chronique.c==3].to_list()

In [30]:
df_das

,type_hosp,cage,sexe,diag,das,nb_das,nb_situations,chronic
0,HC,[0-18[,1,A020,E86,191,11,A
1,HC,[0-18[,1,A020,E8718,107,11,A
2,HC,[0-18[,1,A020,E8768,31,11,A
3,HC,[0-18[,1,A020,K625,23,11,A
4,HC,[0-18[,1,A020,R104,24,11,A
...,...,...,...,...,...,...,...,...
399324,HP,[18-30[,2,E755,K760,15,7,A
399325,HP,[18-30[,2,E755,R160,15,7,A
399326,HP,[18-30[,2,E755,R740,15,7,A
399327,HP,[18-30[,2,E755,Z8701,15,7,A


In [70]:
## Define DAS as chronical or acute
## Keep only DAS from different ICD10 chapter of the DAS, except for Cancer metastases
## Don't take Z codes
## C = chronic, A= acute (possible complications)
df_das = df_das[ ( (df_das.diag.str.slice(0,2)!=df_das.das.str.slice(0,2)) | (df_das.das.isin(codes_cancer_meta)) )  & (df_das.das.str.slice(0,1)!= "Z" ) ]
df_das = df_das.assign(chronic = np.where(df_das.das.isin(icd10_chronic),"C","A"))

In [47]:
## Prep DAS dataframe with a higher granularity in case we don't find any associated with DP
df_das_comp = df_das.assign(categ = df_das.diag.str.slice(0,3)).groupby(["sexe","chronic","categ"]).aggregate(nb_situations = ("nb_situations","sum")).reset_index()

In [ ]:
def assign_das_to_situation(situation):


In [63]:
situation = df_situations.iloc[1000,:]

In [72]:
situation

,type_hosp,cage,sexe,diag,das,nb_das,nb_situations,chronic
61461,HC,[60-70[,1,C040,C770,75,8,C
61462,HC,[60-70[,1,C040,E43,28,8,A
61463,HC,[60-70[,1,C040,I10,75,8,A
61464,HC,[60-70[,1,C040,J448,25,8,A
61465,HC,[60-70[,1,C040,T810,25,8,A
61466,HC,[60-70[,1,C040,T8138,26,8,A


array([1])

In [94]:
selection = df_das[(df_das.cage == situation.cage) & (df_das.diag==situation.diag) & (df_das.sexe==situation.sexe) & (df_das.type_hosp==situation.type_hosp) ]
if len(selection) == 0:
  selection = df_das[(df_das.cage == situation.cage) & (df_das.diag==situation.diag) & (df_das.sexe==situation.sexe)  ]
  if len(selection) == 0:
    selection = df_das_comp[(df_das_comp.categ==situation.diag[0:2]) & (df_das_comp.sexe==situation.sexe) ]

## Between 0 and 3 comorbidities
nb_co = np.random.randint(3, size=1)
print(nb_co)
selection[selection.chronic=="C"].sample(n=nb_co, replace=False, weights="nb_das")
## Between 0 and 2 complications

[2]


ValueError: Cannot take a larger sample than population when 'replace=False'

In [74]:
selection

,type_hosp,cage,sexe,diag,das,nb_das,nb_situations,chronic
61461,HC,[60-70[,1,C040,C770,75,8,C
61462,HC,[60-70[,1,C040,E43,28,8,A
61463,HC,[60-70[,1,C040,I10,75,8,A
61464,HC,[60-70[,1,C040,J448,25,8,A
61465,HC,[60-70[,1,C040,T810,25,8,A
61466,HC,[60-70[,1,C040,T8138,26,8,A


,sexe,categ,nb_situations


In [ ]:
# Keep chronic diseases and non Z code as DAS
df_das

In [ ]:
# df_actes[df_actes["racine"].str[2:3].isin(["C", "K"])].drop_duplicates(subset="acte")

In [ ]:
# df_situations[((df_situations["racine"].isin(racine_deces)) | (df_situations["mode_sortie"] == "DECES")) & (df_situations["diag"].isin(codes_cancer))]

In [ ]:
# Keep cancer as DP
df_das_cancer = df_das[df_das["diag"].isin(codes_cancer)].reset_index(drop=True).drop(columns=["nb_situations"])

In [ ]:
# Keep chronic diseases and non Z code as DAS
chronique_keep = list(df_chronique[df_chronique.iloc[:, 1].isin([1, 2, 3]) &
                                   (df_chronique.iloc[:, 0].str[0] != "Z")].iloc[:, 0])

In [ ]:
df_das_cancer = df_das_cancer[df_das_cancer["das"].isin(chronique_keep)].reset_index(drop=True)

In [ ]:
df_das_cancer

,type_hosp,cage,sexe,diag,das,nb_das
0,HC,[0-18[,1,C402,C492,115
1,HC,[0-18[,1,C402,C780,109
2,HC,[0-18[,1,C402,C795,64
3,HC,[0-18[,1,C741,C795,67
4,HC,[0-18[,1,C749,C770,23
...,...,...,...,...,...,...
11022,HP,[0-18[,2,C64,C780,39
11023,HP,[0-18[,2,C716,C793,13
11024,HP,[0-18[,2,C716,C794,35
11025,HP,[0-18[,2,C910,C793,28


In [ ]:
df_das_cancer["sum"] = df_das_cancer.groupby(["type_hosp", "cage", "sexe", "diag"])["nb_das"].transform("sum")

In [ ]:
df_das_cancer

,type_hosp,cage,sexe,diag,das,nb_das,sum
0,HC,[0-18[,1,C402,C492,115,288
1,HC,[0-18[,1,C402,C780,109,288
2,HC,[0-18[,1,C402,C795,64,288
3,HC,[0-18[,1,C741,C795,67,67
4,HC,[0-18[,1,C749,C770,23,497
...,...,...,...,...,...,...,...
11022,HP,[0-18[,2,C64,C780,39,311
11023,HP,[0-18[,2,C716,C793,13,83
11024,HP,[0-18[,2,C716,C794,35,83
11025,HP,[0-18[,2,C910,C793,28,130


In [ ]:
df_das_cancer["das_first_letter"] = df_das_cancer["das"].str[0]

In [ ]:
df_das_cancer

,type_hosp,cage,sexe,diag,das,nb_das,sum,das_first_letter
0,HC,[0-18[,1,C402,C492,115,288,C
1,HC,[0-18[,1,C402,C780,109,288,C
2,HC,[0-18[,1,C402,C795,64,288,C
3,HC,[0-18[,1,C741,C795,67,67,C
4,HC,[0-18[,1,C749,C770,23,497,C
...,...,...,...,...,...,...,...,...
11022,HP,[0-18[,2,C64,C780,39,311,C
11023,HP,[0-18[,2,C716,C793,13,83,C
11024,HP,[0-18[,2,C716,C794,35,83,C
11025,HP,[0-18[,2,C910,C793,28,130,C


In [ ]:
df_das_cancer = df_das_cancer.groupby(
    ["type_hosp", "cage", "sexe", "diag", "das_first_letter", "sum"]
    )[["das", "nb_das"]].apply(
        lambda g: dict(zip(g["das"], g["nb_das"]))
        ).reset_index(name="das_dict")

In [ ]:
df_das_cancer

,type_hosp,cage,sexe,diag,das_first_letter,sum,das_dict
0,HC,[0-18[,1,C402,C,288,"{'C492': 115, 'C780': 109, 'C795': 64}"
1,HC,[0-18[,1,C741,C,67,{'C795': 67}
2,HC,[0-18[,1,C749,C,497,"{'C770': 23, 'C771': 24, 'C772': 65, 'C780': 2..."
3,HC,[0-18[,1,C910,E,22,{'E109': 22}
4,HC,[0-18[,2,C402,C,190,"{'C492': 52, 'C780': 91, 'C795': 47}"
...,...,...,...,...,...,...,...
1715,HP,[80-[,2,D462,D,66,{'D469': 35}
1716,HP,[80-[,2,D464,D,35,{'D469': 35}
1717,HP,[80-[,2,D469,C,141,{'C920': 51}
1718,HP,[80-[,2,D469,D,141,"{'D462': 37, 'D638': 31}"


In [ ]:
# df_das_cancer.iloc[0]["das"]

In [ ]:
# df_letter_counts = (
#     df_das_cancer
#     .groupby(["type_hosp", "cage", "sexe", "diag"])["das_first_letter"]
#     .nunique()
#     .reset_index(name="letter_count")
# )

In [ ]:
# df_letter_counts

In [ ]:
# df_das_cancer = df_das_cancer.merge(df_letter_counts, on=["type_hosp", "cage", "sexe", "diag"]).drop(columns=["das_first_letter"])

In [ ]:
# df_das_cancer.iloc[0]["das"]

In [ ]:
# df_das_cancer

In [ ]:
# Keep cancer as DP
df_cancer_situations = df_situations[df_situations["diag"].isin(codes_cancer)].reset_index(drop=True).drop(columns=["gp", "ga", "nb", "dms"])

In [ ]:
df_cancer_situations

,diag,cage,sexe,mode_entree,mode_sortie,type_hosp,racine,libelle_racine
0,C01,[40-50[,1,DOMICILE,DOMICILE,HP,28Z07,"Chimiothérapie pour tumeur, en séances"
1,C01,[50-60[,1,DOMICILE,DOMICILE,HC,03M07,"Tumeurs malignes des oreilles, du nez, de la g..."
2,C01,[50-60[,1,DOMICILE,DOMICILE,HC,17M06,Chimiothérapie pour autre tumeur
3,C01,[50-60[,1,DOMICILE,DOMICILE,HP,03K03,Séjours comprenant une endoscopie oto-rhino-la...
4,C01,[50-60[,1,DOMICILE,DOMICILE,HP,28Z07,"Chimiothérapie pour tumeur, en séances"
...,...,...,...,...,...,...,...,...
8162,D469,[80-[,1,URGENCES,DOMICILE,HC,17M16,Hémopathies myéloïdes chroniques
8163,D469,[80-[,2,DOMICILE,DOMICILE,HC,17M16,Hémopathies myéloïdes chroniques
8164,D469,[80-[,2,DOMICILE,DOMICILE,HP,28Z14,"Transfusions, en séances"
8165,D469,[80-[,2,URGENCES,DOMICILE,HC,17M16,Hémopathies myéloïdes chroniques


In [ ]:
def sample_das(case): ## Problem: There are ICD codes that don't have any `das`
  das = df_das_cancer[(df_das_cancer["type_hosp"] == case["type_hosp"]) &\
                             (df_das_cancer["sexe"] == case["sexe"]) &\
                             (df_das_cancer["cage"] == case["cage"]) &\
                             (df_das_cancer["diag"] == case["diag"])].reset_index(drop=True)
  if len(das) > 0:
    nb_letters = das.iloc[0]["letter_count"]
    nb_das = min(np.random.randint(low=0, high=nb_letters+1), 4)

    list_das = []
    if nb_das > 0:
      letters = np.random.choice(nb_letters, size=nb_das, replace=False)
      for letter in letters:
        list_das.append(str(np.random.choice(das.iloc[letter]["das"].copy())))

    return list_das

  else:
    return None

In [ ]:
df_cancer_situations["das"] = df_cancer_situations.apply(sample_das, axis=1)

In [ ]:
# df_cancer_situations.iloc[1]["das"]

In [ ]:
# df_cancer_situations = df_cancer_situations.dropna()

In [ ]:
df_cancer_situations

In [ ]:
df_synonym = pd.read_csv(path_ref + "cim_synonymes.csv").dropna()

In [ ]:
df_synonym

In [ ]:
# df_synonym = df_synonym[df_synonym["tp"]>0].reset_index(drop=True)

In [ ]:
df_synonym = df_synonym[df_synonym["code_count"]>0].reset_index(drop=True)

In [ ]:
df_synonym

In [ ]:
df_synonym = df_synonym.groupby(["code"]).agg(
    dictionary_keys=('dictionary_keys', lambda x: list(x)),
    count=('dictionary_keys', 'size')
    ).reset_index()

In [ ]:
# df_synonym.iloc[0]["dictionary_keys"]

In [ ]:
df_synonym

In [ ]:
df_synonym = df_synonym[df_synonym["count"]>0].reset_index(drop=True)

In [ ]:
df_synonym

In [ ]:
df_synonym["dictionary_keys_temp"] = df_synonym["dictionary_keys"].copy()

In [ ]:
df_synonym

In [ ]:
df_icd_official

In [ ]:
# def sample_synonym(case, type="das"):
#   if type == "das":
#     codes = case["das"]
#   else:
#     codes = [case["diag"]]

#   list_synonyms = {}
#   if codes is not None:
#     if len(codes) > 0:
#       for code in codes:
#         if len(df_synonym[df_synonym["code"] == code]) == 0:
#           continue
#         else:
#           if len(df_synonym[df_synonym["code"] == code].iloc[0]["dictionary_keys_temp"]) == 0:
#             df_synonym[df_synonym["code"] == code].at[0, "dictionary_keys_temp"] = df_synonym[df_synonym["code"] == code].iloc[0]["dictionary_keys"].copy()

#           synonyms = df_synonym[df_synonym["code"] == code].iloc[0]["dictionary_keys_temp"]
#           synonym = str(np.random.choice(synonyms))
#           list_synonyms[synonym] = df_icd_official[df_icd_official["code"]==code].iloc[0]["libelle"]
#           df_synonym[df_synonym["code"] == code].at[0, "dictionary_keys_temp"].remove(synonym)

#   return list_synonyms

In [ ]:
def sample_synonym(case, type="das"): # Problem: There are some ICD codes that don't have any synonyms
  if type == "das":
    codes = case["das"]
  else:
    codes = [case["diag"]]

  list_synonyms = {}
  if codes is not None:
    if len(codes) > 0:
      for code in codes:
        row_synonym = df_synonym[df_synonym["code"] == code]
        if len(row_synonym) == 0:
          continue
        else:
          idx = row_synonym.index[0]
          row_synonym = row_synonym.iloc[0]

          if len(row_synonym["dictionary_keys_temp"]) == 0:
            synonyms = row_synonym["dictionary_keys"].copy()
          else:
            synonyms = row_synonym["dictionary_keys_temp"].copy()

          synonym = str(np.random.choice(synonyms))
          list_synonyms[synonym] = df_icd_official[df_icd_official["code"]==code].iloc[0]["libelle"]
          synonyms.remove(synonym)
          df_synonym.at[idx, "dictionary_keys_temp"] = synonyms

  return list_synonyms

In [ ]:
df_cancer_situations["das_synonyms"] = df_cancer_situations.apply(sample_synonym, axis=1)
df_cancer_situations["diag_synonyms"] = df_cancer_situations.apply(lambda x : sample_synonym(x, type="diag"), axis=1)

In [ ]:
# df_cancer_situations.iloc[1]["das_synonyms"]

In [ ]:
# df_cancer_situations.iloc[1]["diag_synonyms"]

In [ ]:
# df_cancer_situations.iloc[1]["das"]

In [ ]:
df_cancer_situations

In [ ]:
def situation(case):
    situa = ""
    code = 0

    if case["racine"] in racine_chimio and case["type_hosp"]  == "HP" :
      situa = "Prise en charge en hospitalisation de jour pour cure de chimiothérapie"
      code = 1

    elif case["racine"] in racine_chimio and case["type_hosp"]  == "HC":
      situa = "Prise en charge en hospitalisation complète pour cure de chimiothérapie"
      code = 2

    elif case["racine"] in racine_radio and case["type_hosp"]  == "HP":
      situa = "Prise en charge en hospitalisation de jour pour séance de radiothérapie"
      code = 3

    elif case["racine"] in racine_radio and case["type_hosp"]  == "HC":
      situa = "Prise en charge en hospitalisation complète pour réalisation du traitment de radiothérapie"
      code = 4

    elif case["racine"] in racine_greffe:
      situa = "Prise en charge pour " + case["libelle_racine"].lower()
      code = 5

    elif case["racine"] in racine_transfusion :
      situa = "Prise en charge pour " + case["libelle_racine"].lower()
      code = 6

    elif case["racine"] in racine_aphereses :
      situa = "Prise en charge pour " + case["libelle_racine"].lower()
      code = 7

    elif case["racine"] in racine_soins_palliatifs :
      actes = df_actes[(df_actes["racine"] == case["racine"]) &\
                       (df_actes["type_hosp"] == case["type_hosp"]) &\
                       (df_actes["sexe"] == case["sexe"]) &\
                       (df_actes["diag"] == case["diag"])]

      if len(actes) > 0:
        situa = "Prise en charge pour soins palliatifs avec " + actes.sample(1).iloc[0].copy()["libelle_long"].lower()
      else:
        situa = "Prise en charge pour soins palliatifs"

      code = 8

    elif case["racine"] in racine_soins_stomies:
      situa = "Prise en charge pour " + case["libelle_racine"].lower()
      code = 9

    elif (case["racine"] in racine_deces) | (case["mode_sortie"] == "DECES"):
      situa =  "Hospitalisation au cours de laquelle le patient est décédé"
      code = 10

    elif case["racine"][2:3] in ["C","K"] and case["type_hosp"]  == "HP" :
      situa =  "Prise en charge en chirugie ambulatoire pour "
      actes = df_actes[(df_actes["racine"] == case["racine"]) &\
                       (df_actes["type_hosp"] == case["type_hosp"]) &\
                       (df_actes["sexe"] == case["sexe"]) &\
                       (df_actes["diag"] == case["diag"])]

      if len(actes) > 0:
        situa += actes.sample(1).iloc[0].copy()["libelle_long"].lower()

      code = 11

    elif case["racine"][2:3] in ["C","K"] and case["type_hosp"]  == "HC" :
      situa =  "Prise en charge chirugicale en hospitalisation complète pour "
      actes = df_actes[(df_actes["racine"] == case["racine"]) &\
                       (df_actes["type_hosp"] == case["type_hosp"]) &\
                       (df_actes["sexe"] == case["sexe"]) &\
                       (df_actes["diag"] == case["diag"])]

      if len(actes) > 0:
        situa += actes.sample(1).iloc[0].copy()["libelle_long"].lower()

      code = 12

    else :
      option = np.random.choice(3, p=[0.4, 0.3, 0.3])
      if option == 0:
        situa = "Première hospitalisation pour découverte de cancer" # 40%
        code = 13
      elif option == 1:
        situa = "Première diagnostique et thérapeutique dans le cadre d'une rechute du cancer après traitement" # 30%
        code = 14
      else:
        situa =  "Hospitalisation pour bilan et surveillance du cancer" # 30%
        code = 15

    return (situa, code)

In [ ]:
df_cancer_situations[["situation", "code_situa"]] = df_cancer_situations.apply(situation, axis=1, result_type="expand")

In [ ]:
df_cancer_situations

In [ ]:
# df_cancer_situations = df_cancer_situations[df_cancer_situations["diag_synonyms"].apply(lambda x: len(x) > 0) |
#                                             df_cancer_situations["das_synonyms"].apply(lambda x: len(x) > 0)].reset_index(drop=True)

In [ ]:
# type(df_cancer_situations["diag"].iloc[1])

In [ ]:
df_cancer_situations = df_cancer_situations[df_cancer_situations["diag_synonyms"].apply(lambda x: len(x) > 0)].reset_index(drop=True)

In [ ]:
df_cancer_situations

In [ ]:
df_cancer_situations.to_csv(path_ref + "cancer_situations_complete.csv")

In [ ]:
# df_cancer_situations[(df_cancer_situations["racine"].isin(racine_deces)) | (df_cancer_situations["mode_sortie"] == "DECES")].drop_duplicates(subset="situation")